## Preparation

In [1]:
%%capture
!conda install -c pykaldi pykaldi -y

In [2]:
%cd /tmp

/tmp


In [3]:
!git clone https://github.com/pytorch/fairseq/

Cloning into 'fairseq'...
remote: Enumerating objects: 28172, done.
remote: Counting objects: 100% (586/586), done.
remote: Compressing objects: 100% (337/337), done.
remote: Total 28172 (delta 308), reused 464 (delta 236), pack-reused 27586
Receiving objects: 100% (28172/28172), 11.87 MiB | 17.98 MiB/s, done.
Resolving deltas: 100% (21131/21131), done.


In [4]:
%%capture
!pip install git+https://github.com/pytorch/fairseq/

In [5]:
!git clone https://github.com/kpu/kenlm

Cloning into 'kenlm'...
remote: Enumerating objects: 13824, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 13824 (delta 76), reused 92 (delta 45), pack-reused 13687
Receiving objects: 100% (13824/13824), 5.49 MiB | 10.38 MiB/s, done.
Resolving deltas: 100% (7956/7956), done.


In [6]:
%%capture
!apt-get -y install libeigen3-dev liblzma-dev zlib1g-dev libbz2-dev

In [7]:
%%capture
%cd kenlm
!mkdir build
%cd build
!cmake ..
!make -j 4
%cd /tmp

In [8]:
import os
os.environ['PATH'] = f"{os.environ['PATH']}:/tmp/kenlm/build/bin/"
os.environ['FAIRSEQ_ROOT'] = '/tmp/fairseq'

In [9]:
%cd fairseq/examples/wav2vec/unsupervised

/tmp/fairseq/examples/wav2vec/unsupervised


In [10]:
# Needed to see what's going wrong
os.environ['HYDRA_FULL_ERROR'] = '1'

## GAN

In [11]:
!ls /kaggle/input/wav2vec-u-cv-swedish-audio/

CLUS128		    precompute_pca512_cls128_mean	  train.npy
__notebook__.ipynb  precompute_pca512_cls128_mean_pooled  train.phn
__output__.json     test.lengths			  train.tsv
__results__.html    test.ltr				  train.wrd
custom.css	    test.npy				  valid.lengths
dict.test	    test.phn				  valid.ltr
dict.train	    test.tsv				  valid.npy
dict.valid	    test.wrd				  valid.phn
pca		    train.lengths			  valid.tsv
precompute_pca512   train.ltr				  valid.wrd


In [12]:
!PREFIX=w2v_unsup_gan_xp fairseq-hydra-train -m --config-dir config/gan --config-path . --config-name w2vu task._name='unpaired_audio_text' task.data=/kaggle/input/wav2vec-u-cv-swedish-audio/precompute_pca512_cls128_mean_pooled/ task.text_data=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/ task.kenlm_path=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/lm.phones.filtered.04.bin checkpoint.no_epoch_checkpoints=false checkpoint.save_dir=/kaggle/working 'common.seed=range(0,5)'

[2021-05-30 19:39:17,860][HYDRA] Launching 5 jobs locally
[2021-05-30 19:39:17,860][HYDRA] 	#0 : task._name=unpaired_audio_text task.data=/kaggle/input/wav2vec-u-cv-swedish-audio/precompute_pca512_cls128_mean_pooled/ task.text_data=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/ task.kenlm_path=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/lm.phones.filtered.04.bin checkpoint.no_epoch_checkpoints=False checkpoint.save_dir=/kaggle/working common.seed=0
[2021-05-30 19:39:20,222][fairseq_cli.train][INFO] - {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': 'json', 'log_file': None, 'tensorboard_logdir': 'tb', 'wandb_project': None, 'azureml_logging': False, 'seed': 0, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': True, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_l

In [13]:
#!bash rungan.sh